<a href="https://colab.research.google.com/github/sajidmahmud69/Machine-Learning/blob/main/Colab_Webcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab: Access Webcam for Images and Video

This will use OpenCV's Haar Cascade to do fave detection on our webcam image and video

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL

# standard library
import io
import html
import time

# Helper  Functions

Convert image files from javascript to opencv

In [ ]:
def js_to_image (js_reply):
  """
  Params:
          js_reply: Javascript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """

  # decode base64 image
  image_bytes = b64decode (js_reply.split(',')[1])

  # convert bytes to numpy array
  jpg_as_np = np.frombuffer (image_bytes, dtype = uint8)

  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode (jpg_as_np, flags = 1)

  return img



In [ ]:
# Function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed in video stream

def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) conatining rectangle to overlay on video stream
  Returns:
          bytes: Base64 image byte string
  """

  # Convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()

  # format bbox into png for return
  bbox_PIL.save (iobuf, format = 'png')

  # format return string

  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

# Haar Cascade Classifier
For this tutorial we will run a simple object detection algorithm called Haar Cascade on our images and video fetched from our webcam.
OPenCV has a pre trained Haar Cascade face detection model.

In [ ]:
face_cascade = cv2.CascadeClassifier (cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

# Webcam images

Running Code om images taken from webcam is fairly straight forward. We will utilize code within Google Colab's Code Snippets that has a variety of useful code functions to peform various tasks.

We will be using the code snippet for the **Camera Capture** to utilize the computer's webcam.

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get the photo data
  data = eval_js('takePhoto({})'.format(quality))

  # get OpenCV image format
  img = js_to_image (data)

  # grayscale img

  gray = cv2.cvtColor (img, cv2.COLOR_RGB2GRAY)
  print (gray.shape)

  # get face boudning box cooridantes using haar cascade
  faces = face_cascade_detectMultiScale (gray)

  # draw face bounding box on image

  for (x,y,w,h) in faces:
    img = cv2.rectangle (img, (x,y), (x+w, y+h), (255, 0, 0), 2)
  
  # save image
  cv2.imwrite (filename, img)
  

In [ ]:

try:
  filename = take_photo('photo_jpg')
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

<IPython.core.display.Javascript object>

name 'uint8' is not defined
